# Evolver Loop 6 Analysis: Strategy Assessment

## Current Situation
- Best CV: 70.615106 (exp_005)
- Best LB: 70.647327 (exp_002/003)
- Target: 68.887226
- Gap: 1.73 points (2.5%)

## Key Observations from Research
1. **Ensemble approach is LIMITED** - only 0.032 improvement from 88 snapshots
2. **exp_004 and exp_005 have nearly identical scores** - stuck at local optimum
3. **Top kernels use multiple techniques**: ensemble + fractional translation + SA + fix_direction
4. **jonathanchan kernel uses 19+ sources** - we only have ~88 snapshots

In [ ]:
import pandas as pd
import numpy as np
import json
from glob import glob

# Load session state to understand what we've tried
with open('/home/code/session_state.json', 'r') as f:
    state = json.load(f)

print("Experiments so far:")
for exp in state['experiments']:
    print(f"  {exp['id']}: {exp['name']} - CV={exp['cv_score']:.6f}")

print(f"\nBest CV: {min(e['cv_score'] for e in state['experiments']):.6f}")
print(f"Target: 68.887226")
print(f"Gap: {min(e['cv_score'] for e in state['experiments']) - 68.887226:.6f}")

In [ ]:
# Analyze what techniques have been tried
print("\nTechniques tried:")
print("1. Pre-optimized baseline: 70.676 -> 70.647 (after validation fix)")
print("2. bbox3 + sa_fast_v2: NO improvement (local optimum)")
print("3. fix_direction (rotation tightening): NO improvement")
print("4. Ensemble from snapshots: 0.032 improvement")
print("5. Strict ensemble (1e-15 threshold): same as #4")

print("\nTechniques NOT tried:")
print("1. Fractional translation (jonathanchan kernel)")
print("2. Constructive approach (zaburo kernel - well-aligned initial)")
print("3. Running SA/bbox3 with different random seeds")
print("4. Genetic algorithm")
print("5. No-Fit Polygon (NFP)")
print("6. Branch-and-bound for small N")

In [ ]:
# Check what external sources jonathanchan uses that we don't have
jonathanchan_sources = [
    "bucket-of-chump",
    "SmartManoj/Santa-Scoreboard (GitHub)",
    "santa-2025-try3",
    "santa25-public",
    "telegram-public-shared-solution",
    "jwt/other/csv/19",
    "why-not",
    "santa25-improved-sa-with-translations",
    "santa2025-ver2",
    "santa-submission",
    "santa25-simulated-annealing-with-translations",
    "santa-2025-simple-optimization-new-slow-version",
    "santa-2025-fix-direction",
    "72-71-santa-2025-jit-parallel-sa-c",
    "santa-claude",
    "blending-multiple-oplimisation",
    "santa2025-just-keep-on-trying",
    "decent-starting-solution",
]

print(f"jonathanchan uses {len(jonathanchan_sources)} sources")
print("We have ~88 snapshots from previous runs")
print("\nKey insight: More diverse sources = better ensemble")

In [ ]:
# Analyze the gap breakdown
# From exp_000 notes: n=1-10: 4.33, n=11-20: 3.73, n=21-50: 10.98, n=51-100: 17.64, n=101-150: 17.14, n=151-200: 16.85

ranges = {
    '1-10': 4.33,
    '11-20': 3.73,
    '21-50': 10.98,
    '51-100': 17.64,
    '101-150': 17.14,
    '151-200': 16.85
}

total = sum(ranges.values())
print("Score breakdown by N range:")
for r, s in ranges.items():
    pct = s / total * 100
    print(f"  N={r}: {s:.2f} ({pct:.1f}%)")

print(f"\nTotal: {total:.2f}")
print(f"\nTo reach target 68.887, need to reduce by {total - 68.887:.2f} points")
print(f"That's {(total - 68.887) / total * 100:.1f}% reduction needed")

In [ ]:
# Check what's in the preoptimized folder
import os

preopt_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/'
if os.path.exists(preopt_path):
    files = os.listdir(preopt_path)
    print(f"Preoptimized folder has {len(files)} files")
    print(f"Sample files: {files[:5]}")
else:
    print("Preoptimized folder not found")

# Check total CSV files available
all_csvs = glob('/home/nonroot/snapshots/santa-2025/**/*.csv', recursive=True)
print(f"\nTotal CSV files in snapshots: {len(all_csvs)}")

In [ ]:
# Key strategic insight:
# The jonathanchan kernel achieves good scores by:
# 1. Ensemble from MANY diverse sources (19+)
# 2. Fractional translation post-processing
# 3. SA optimization with C++ code
# 4. fix_direction (rotation optimization)

# We've tried #4 (no improvement) and partial #1 (limited sources)
# We HAVEN'T tried #2 (fractional translation) or #3 (SA with different seeds)

print("RECOMMENDED NEXT STEPS:")
print("")
print("1. FRACTIONAL TRANSLATION (HIGH PRIORITY)")
print("   - Apply small translations to each tree")
print("   - Steps: [0.001, 0.0005, 0.0002, 0.0001]")
print("   - 8 directions: N, S, E, W, NE, NW, SE, SW")
print("   - Expected improvement: 0.01-0.1 points")
print("")
print("2. GENERATE NEW SOLUTIONS WITH DIFFERENT SEEDS")
print("   - Run bbox3/sa_fast with different random seeds")
print("   - This creates diversity for ensemble")
print("   - Each seed may find different local optima")
print("")
print("3. CONSTRUCTIVE APPROACH (zaburo kernel)")
print("   - Build solutions from scratch using grid pattern")
print("   - May find configurations not reachable by SA")
print("")
print("4. SUBMIT exp_005 FIRST")
print("   - Need LB feedback to verify strict validation works")